In [1]:
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import os
import json
import numpy as np

In [2]:
pre21 = pd.read_csv('./전처리/wi-fi/path/presence_path_2021.csv')

In [3]:
pre22 = pd.read_csv('./전처리/wi-fi/path/presence_path_2022.csv')

In [4]:
# 용량 줄이기
# 데이터 범위 체크 후, 최적화된 타입 적용
def reduce_mem_usage(df):
    
    # 현재 메모리 사용량
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터 프레임의 메모리 사용량 : {:.2f} MB'.format(start_mem))
    
    # 컬럼 하나씩 확인 후 적용
    for col in tqdm(df.columns[1:]):
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    # 타입변경 후 메모리 사용량        
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 메모리 사용량 : {:.2f} MB'.format(end_mem))
    print('감소율 : {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
# 데이터프레임 타입변경
def df_astype(df):
    reduce_mem_usage(df)
    df['STARTTIMESTAMP'] = pd.to_datetime(df['STARTTIMESTAMP'])
    df['ENDTIMESTAMP'] = pd.to_datetime(df['ENDTIMESTAMP'])

In [6]:
df_astype(pre)

데이터 프레임의 메모리 사용량 : 2705.92 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 1646.35 MB
감소율 : 39.2%


In [7]:
df_astype(pre22)

데이터 프레임의 메모리 사용량 : 1765.12 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 826.17 MB
감소율 : 53.2%


In [8]:
# 동일한 위경도값을 보유한 포인트를 1개의 개소로 측정

# 위경도 csv 파일 읽기
wk = pd.read_csv('./전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])


# MAC 코드화 딕셔너리 불러오기
with open('mac_code.json', 'r') as f :
    mac_code = json.load(f, encoding='utf-8')

# key값과 values값 위치 변경
mac_code = dict(zip(mac_code.values(),mac_code.keys()))

In [9]:
mac_code

{1: '7C573CCF4D0C',
 2: '7C573CCF4AC0',
 3: '7C573CCF457A',
 4: '7C573CCF4F62',
 5: '7C573CCF4B2E',
 6: '7C573CCF4D94',
 7: '7C573CCF4A6E',
 8: '7C573CCF43EE',
 9: '7C573CCF4D6A',
 10: '7C573CCF4E8C',
 11: '7C573CCF4EF2',
 12: '7C573CCF4BD4',
 13: '7C573CCF47E6',
 14: '7C573CCF47D0',
 15: '7C573CCF44C0',
 16: '7C573CCFB76A',
 17: '7C573CCF4398',
 18: '7C573CCF4D54',
 19: '7C573CCF4AD6',
 20: '7C573CCF4ECC',
 21: '7C573CCF4AAE',
 22: '7C573CCF4352',
 23: '7C573CCF484E',
 24: '7C573CCF4C04',
 25: '7C573CCF4C3E',
 26: '7C573CCF4EA6',
 27: '7C573CCF4740',
 28: '7C573CCF49AA',
 29: '7C573CCF4CE8',
 30: '7C573CCF4AE6',
 31: '7C573CCF4B36',
 32: '7C573CCF49F8',
 33: '7C573CCF4A94',
 34: '7C573CCF4D10',
 35: '7C573CCF4E28',
 36: '7C573CCF4BDA',
 37: '7C573CCF454E',
 38: '7C573CCF4D74',
 39: '7C573CCF4F88',
 40: 'D015A6C7F4F2',
 41: '7C573CCF43DE',
 42: '7C573CCF45BA',
 43: '7C573CCF44DA',
 44: '7C573CCF45F4',
 45: '7C573CCF470A',
 46: '7C573CCF4518',
 47: '7C573CCF4302',
 48: '7C573CCF4B80',
 

In [11]:
# 사용되지 않은 Wi-Fi, 목록에서 제거
for n in wk.index:
    if n not in mac_code.values():
        wk.drop(n, axis=0, inplace=True)

# 위도로 정렬 (같은 위도 묶기)
wk.sort_values('위도', inplace=True)

In [13]:
# 이전 Mac 주소를 담을 변수
f_mac = None

# 동일한 위경도를 보유한 Mac 주소 리스트
repeat = []

# repeat 리스트를 모은 리스트
repeat_all = []

for n in wk.index:
    # 첫행 Skip
    if n == wk.iloc[0].name:
        f_mac = n
        continue
    # 이전 행 (위도, 경도)와 같다면 append
    if (wk.loc[f_mac]['위도'] == wk.loc[n]['위도']) and (wk.loc[f_mac]['경도'] == wk.loc[n]['경도']):
        repeat.append(f_mac)
        repeat.append(n)
    
    # 전체 리스트 
    else:
        # 빈 리스트는 무시
        if len(repeat) == 0:
            f_mac = n
            continue
        repeat_all.append(list(set(repeat)))
        repeat = []
    f_mac = n

In [15]:
with open('mac_code.json', 'r') as f :
    mac_code = json.load(f, encoding='utf-8')

repeat_num_all = []
repeat_num = []
for n in repeat_all:
    for i in range(len(n)):
        repeat_num.append(mac_code[n[i]])
    repeat_num_all.append(repeat_num)
    repeat_num = []
    
    
# key값과 values값 위치 변경
mac_code = dict(zip(mac_code.values(),mac_code.keys()))


for n in repeat_num_all:
    for i in range(len(n)):
        if i == 0:
            continue
        mac_code[n[i]] = mac_code[n[0]]

# 연도별

In [20]:
# 사용조건 연도별 파일이어야함

def year_split(df, year, encoding='cp949'):
    # 년도 폴더 생성
    try:
        directory_01 = './전처리/wi-fi/path/' + f'{year}'
        if not os.path.exists(directory_01):
            os.makedirs(directory_01)
    except OSError:
        print ('Error: Creating directory. ' +  directory_01)
    
    # 재자리에 있는 사용자 제외
    # 같은 위경도 포인트 병합
    #----------------------------------------------------------------------------------  
    df = df[df['Next_MAC']>0]
    df.reset_index(drop=True, inplace=True)
    for x in tqdm(range(len(df))):
        df['MAC_add'][x] = mac_code[df['MAC_add'][x]]
        df['Next_MAC'][x] = mac_code[df['Next_MAC'][x]]
    #----------------------------------------------------------------------------------



    # 월별 csv 파일 저장    
    df.to_csv(f'{directory_01}/pre_{year}.csv', encoding=encoding, index=False)


    # 월별 폴더 생성
    try:
        directory_02 = directory_01 + f'/개소별'
        if not os.path.exists(directory_02):
            os.makedirs(directory_02)
    except OSError:
        print ('Error: Creating directory. ' +  directory_02)        


    # Wi-Fi 개소 list
    df_mac_li = list(set(df['MAC_add']))

    # Wi-Fi 개소별 나누기
    for n in tqdm(df_mac_li):
        small_df = df[df['MAC_add'] == n]
        small_df.to_csv(f'{directory_02}/path_{year}_{n}.csv', encoding=encoding, index=False)

In [21]:
year_split(pre21, 2021)

  0%|          | 0/3255457 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16416\1696898213.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][x] = mac_code[df['MAC_add'][x]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16416\1696898213.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][x] = mac_code[df['Next_MAC'][x]]


  0%|          | 0/53 [00:00<?, ?it/s]

In [22]:
year_split(pre22, 2022)

  0%|          | 0/2030137 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16416\1696898213.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][x] = mac_code[df['MAC_add'][x]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16416\1696898213.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][x] = mac_code[df['Next_MAC'][x]]


  0%|          | 0/53 [00:00<?, ?it/s]

# 연도별_개소_counting

In [23]:
def Year_Path_counting(year, encoding='cp949'):
    
    year_path = f'./전처리/wi-fi/path/{year}/개소별/'
    
    wifi_csv_list = os.listdir(year_path)

    try:
        directory = f'./전처리/wi-fi/path/count/{year}/'
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

    for i in tqdm(wifi_csv_list):
        df = pd.read_csv(year_path + i , encoding=encoding)
        df = df[['MAC_add','Next_MAC']]


        df = pd.DataFrame(df.groupby(['MAC_add','Next_MAC']).value_counts(), columns=['count'])
        df.to_csv(f'{directory}{i}', encoding=encoding)

In [24]:
Year_Path_counting(2021)

  0%|          | 0/53 [00:00<?, ?it/s]

In [25]:
Year_Path_counting(2022)

  0%|          | 0/53 [00:00<?, ?it/s]

# 월별 split

In [26]:
# 사용조건 년도별 파일이어야함

def month_split(DF, year, encoding='cp949'):
    # 년도별 파일의 최대 월 찾기
    length = DF['STARTTIMESTAMP'].dt.month.max()

    # 년도 폴더 생성
    try:
        directory_01 = './전처리/wi-fi/path/' + f'{year}년_월별'
        if not os.path.exists(directory_01):
            os.makedirs(directory_01)
    except OSError:
        print ('Error: Creating directory. ' +  directory_01)
    
    # 월별 나누기
    print('전체 진행도------------------------------------------------------')
    for n in tqdm(range(1, length+1)):
        if n <= 9:
            num = '0' + str(n)
        else:
            num = str(n)

        # 기간설정(월별)
        if n != 12:
            df = DF[(DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S')) &
                    (DF['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{n+1}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S'))]
        else:
            df = DF[DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00', 
                                                              '%Y-%m-%d %H:%M:%S')]
        
        
        #----------------------------------------------------------------------------------  
        df = df[df['Next_MAC']>0]
        df.reset_index(drop=True, inplace=True)
        for x in range(len(df)):
            df['MAC_add'][x] = mac_code[df['MAC_add'][x]]
            df['Next_MAC'][x] = mac_code[df['Next_MAC'][x]]
        #----------------------------------------------------------------------------------
        
        

        # 월별 csv 파일 저장    
        df.to_csv(f'{directory_01}/pre_{year}_{num}월.csv', encoding=encoding, index=False)
        
        
        # 월별 폴더 생성
        try:
            directory_02 = directory_01 + f'/{num}월'
            if not os.path.exists(directory_02):
                os.makedirs(directory_02)
        except OSError:
            print ('Error: Creating directory. ' +  directory_02)        
        

        # Wi-Fi 개소 list
        df_mac_li = list(set(df['MAC_add']))
        
        # Wi-Fi 개소별 나누기
        print(f'{n}월 진행도------------------------------------------------------')
        for n in tqdm(df_mac_li):
            small_df = df[df['MAC_add'] == n]
            small_df.to_csv(f'{directory_02}/path_{year}_{num}월_{n}.csv', encoding=encoding, index=False)

In [27]:
month_split(pre21, 2021)

전체 진행도------------------------------------------------------


  0%|          | 0/12 [00:00<?, ?it/s]

1월 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

2월 진행도------------------------------------------------------


...

  0%|          | 0/53 [00:00<?, ?it/s]

11월 진행도------------------------------------------------------


  0%|          | 0/46 [00:00<?, ?it/s]

12월 진행도------------------------------------------------------


  0%|          | 0/51 [00:00<?, ?it/s]

In [28]:
month_split(pre22, 2022)

전체 진행도------------------------------------------------------


  0%|          | 0/10 [00:00<?, ?it/s]

1월 진행도------------------------------------------------------


0it [00:00, ?it/s]

2월 진행도------------------------------------------------------


...

  0%|          | 0/53 [00:00<?, ?it/s]

9월 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

10월 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

# 월간 Wi-Fi 개소별 counting

In [29]:
def Path_counting(year, encoding='cp949'):
    
    year_path = f'./전처리/wi-fi/path/{year}년_월별/'

    month_dir_list = []
    
    # year_path 디렉토리 내에서 디렉토리만 추출
    for file in os.listdir(year_path):
        X = os.path.join(year_path, file)
        if os.path.isdir(X):
            month_dir_list.append(file)

    
    
    print('전체 진행도------------------------------------------------------')
    
    for n in tqdm(month_dir_list):
        month_path = year_path + n +'/'
        wifi_csv_list = os.listdir(month_path)
        
        try:
            directory = f'./전처리/wi-fi/path/count/{year}년_월별/{n}/'
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)
            
            
        print(f'{n} 진행도------------------------------------------------------')
        
        for i in tqdm(wifi_csv_list):
            df = pd.read_csv(month_path + i , encoding=encoding)
            df = df[['MAC_add','Next_MAC']]
            
            
            df = pd.DataFrame(df.groupby(['MAC_add','Next_MAC']).value_counts(), columns=['count'])
            df.to_csv(f'{directory}{i}', encoding=encoding)

In [30]:
Path_counting(2021)

전체 진행도------------------------------------------------------


  0%|          | 0/12 [00:00<?, ?it/s]

01월 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

02월 진행도------------------------------------------------------


...

  0%|          | 0/53 [00:00<?, ?it/s]

11월 진행도------------------------------------------------------


  0%|          | 0/46 [00:00<?, ?it/s]

12월 진행도------------------------------------------------------


  0%|          | 0/51 [00:00<?, ?it/s]

In [31]:
Path_counting(2022)

전체 진행도------------------------------------------------------


  0%|          | 0/10 [00:00<?, ?it/s]

01월 진행도------------------------------------------------------


0it [00:00, ?it/s]

02월 진행도------------------------------------------------------


...

  0%|          | 0/53 [00:00<?, ?it/s]

09월 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

10월 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]